# 操作Pandas数据

In [1]:
import numpy as np
import pandas as pd

## 通用函数：保留索引
- 对于***一元运算***（像函数与三角函数），这些通用函数将在输出结果中 ***保留索引和列标签***   
- Pandas 是建立在 NumPy 基础之上的，所以任意的 NumPy 的通用函数同样适用于 Pandas 的 Series 和 DataFrame 对象。

In [2]:
rng = np.random.RandomState(42)
ser = pd.Series(rng.randint(0, 10, 4))

ser

0    6
1    3
2    7
3    4
dtype: int64

In [3]:
# 使用 Numpy 的通用函数，得到的是另一个保留索引的对象
np.exp(ser)

0     403.428793
1      20.085537
2    1096.633158
3      54.598150
dtype: float64

In [5]:
df = pd.DataFrame(rng.randint(0, 10, (3, 4)), columns=['A', 'B', 'C', 'D'])
df

,A,B,C,D
0,1,7,5,1
1,4,0,9,5
2,8,0,9,2


In [6]:
# 使用 Numpy 的通用函数，得到的是另一个保留索引的对象
np.sin(df * np.pi / 4)

,A,B,C,D
0,7.071068e-01,-0.707107,-0.707107,0.707107
1,1.224647e-16,0.000000,0.707107,-0.707107
2,-2.449294e-16,0.000000,0.707107,1.000000


## 通用函数：索引对齐
- 当在两个 Series 或 DataFrame 对象上进行 ***二元计算*** 时，Pandas 会在
计算过程中 ***对齐两个对象的索引***。    
- 当处理不完整的数据时，这一点非常方便，我们将在后面的示例中看

In [7]:
# Index alignment in Series

area = pd.Series({
    'Alaska': 1723337,
    'Texas': 695662,
    'California': 423967
},
                 name='area')

population = pd.Series(
    {
        'California': 38332521,
        'Texas': 26448193,
        'New York': 19651127
    },
    name='population')

population / area  # 任意不存在的用 NaN 填充 

Alaska              NaN
California    90.413926
New York            NaN
Texas         38.018740
dtype: float64

In [8]:
area.index | population.index # 结果数组的索引是两个输入数组索引的 并集

Index(['Alaska', 'California', 'New York', 'Texas'], dtype='object')

In [9]:
A = pd.Series([2, 4, 6], index=[0, 1, 2])
B = pd.Series([1, 3, 5], index=[1, 2, 3])
A + B  # 任意不存在的项用 NaN 填充

0    NaN
1    5.0
2    9.0
3    NaN
dtype: float64

In [10]:
A.add(B, fill_value=0) # 不想用 NaN 填充，自定义填充的值

0    2.0
1    5.0
2    9.0
3    5.0
dtype: float64

In [11]:
# Index alignment in DataFrame

A = pd.DataFrame(rng.randint(0, 20, (2, 2)), columns=list('AB'))
A

,A,B
0,11,19
1,2,4


In [12]:
B = pd.DataFrame(rng.randint(0, 10, (3, 3)), columns=list('BAC'))
B

,B,A,C
0,2,6,4
1,8,6,1
2,3,8,1


In [13]:
A + B  # 计算两个 dataFrame 时，同样的对齐索引的规则

,A,B,C
0,17.0,21.0,NaN
1,8.0,12.0,NaN
2,NaN,NaN,NaN


In [14]:
fill = A.stack().mean()
A.add(B, fill_value=fill) # 不使用 NaN 填充， 指定填充值

,A,B,C
0,17.0,21.0,13.0
1,8.0,12.0,10.0
2,17.0,12.0,10.0


> Python 运算符和 Pandas 方法的对应关系       

| Python Operator | Pandas Method(s)                      |
|-----------------|---------------------------------------|
| ``+``           | ``add()``                             |
| ``-``           | ``sub()``, ``subtract()``             |
| ``*``           | ``mul()``, ``multiply()``             |
| ``/``           | ``truediv()``, ``div()``, ``divide()``|
| ``//``          | ``floordiv()``                        |
| ``%``           | ``mod()``                             |
| ``**``          | ``pow()``                             |

## 通用函数：DataFrame与Series之间的运算
与 Numpy 中二维数组与一维数组间的运算规则时一样的

In [15]:
A = rng.randint(10, size=(3, 4))
A

array([[9, 8, 9, 4],
       [1, 3, 6, 7],
       [2, 0, 3, 1]])

In [16]:
A - A[0]

array([[ 0,  0,  0,  0],
       [-8, -5, -3,  3],
       [-7, -8, -6, -3]])

In [17]:
# 默认按 行 运行
df = pd.DataFrame(A, columns=list('QRST'))
df - df.iloc[0]

,Q,R,S,T
0,0,0,0,0
1,-8,-5,-3,3
2,-7,-8,-6,-3


In [19]:
# 指定 广播 的维度
df.subtract(df['R'], axis=0)

,Q,R,S,T
0,1,0,1,-4
1,-2,0,3,4
2,2,0,3,1


In [20]:
halfrow = df.iloc[0, ::2]
halfrow

Q    9
S    9
Name: 0, dtype: int64

In [21]:
# 自动对齐，NaN 填充
df - halfrow

,Q,R,S,T
0,0.0,NaN,0.0,NaN
1,-8.0,NaN,-3.0,NaN
2,-7.0,NaN,-6.0,NaN
